![title](./img/ESPN.png)

In [11]:
# %run ./Competition_mapping.ipynb

--- 🚀 Multi-League Extraction ---
✅ 18 teams captured for fra.1
✅ 18 teams captured for fra.2


In [19]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm.notebook import tqdm
import re
import numpy as np
import datetime
from urllib.parse import urljoin

# --- 1. SETUP ---
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36"}
Hist = []

print(f"📈 Starting historical scrape for {len(Teams)} teams...")

# --- 2. LOOP THROUGH TEAMS ---
for index, row in tqdm(Teams.iterrows(), total=len(Teams)):
    # Construct the results URL for the current season
    # Format: https://www.espn.co.uk/football/team/results/_/id/XXXX/season/2025
    base_url = "https://www.espn.co.uk/"
    # url = urljoin(base_url, row["LinkTeam"]) + "league/" + rljoin(base_url, row["Competition"])
    url = urljoin(base_url, row["LinkTeam"]) + f"league/{row['Competition']}/season/2025"
    
    try:
        response = requests.get(url, headers=headers, timeout=10)
        if response.status_code != 200: continue
        
        soup = BeautifulSoup(response.text, 'lxml')
        
        # ESPN groups results by month/year in 'Table__Title'
        # We need these to attach the correct year to the "Sat, 10 Aug" date strings
        titles = [t.text.split(",")[1].strip() if "," in t.text else "2025" for t in soup.find_all(class_='Table__Title')]
        
        # Find all tables of results
        tables = soup.find_all('div', class_='Table__ScrollerWrapper')
        
        for i, table in enumerate(tables):
            if i >= len(titles): break
            year = titles[i]
            
            for tr in table.find_all('tr', class_='Table__TR--sm'):
                tds = tr.find_all('td')
                if len(tds) < 4: continue
                
                # Check if it's a result (has a score) or a fixture
                score_text = tds[2].text.strip()
                if " - " not in score_text: continue # Skip if no score yet
                Hist.append({
                    "Date": tds[0].text.strip() + " " + year, # e.g., "Sat, 10 Aug 2024"
                    "Home": tds[1].text.strip(),
                    "Score": score_text,
                    "Away": tds[3].text.strip(),
                    "Competition": row["Competition"]
                })
                
    except Exception as e:
        print(f"Error scraping {row['LinkTeam']}: {e}")

# --- 3. CONVERT TO DATAFRAME ---
if Hist:
    print(f"✅ Successfully collected {len(Hist)} historical matches.")
else:
    print("❌ No matches found. Check if the ESPN URLs are correct.")

📈 Starting historical scrape for 36 teams...


  0%|          | 0/36 [00:00<?, ?it/s]

✅ Successfully collected 792 historical matches.


In [21]:
import datetime
import numpy as np
import pandas as pd

# 1. Create DataFrame and drop duplicates
History = pd.DataFrame(Hist).drop_duplicates()

# 2. Safely drop 'Team' ONLY if it exists to prevent the KeyError
if 'Team' in History.columns:
    History = History.drop("Team", axis=1)

# 3. Clean Score and calculate Total Goals
def extract_clean_score(score_str):
    # Regex to find "number - number" patterns, ignoring extra text
    match = re.search(r'(\d+)\s*-\s*(\d+)', str(score_str))
    if match:
        return int(match.group(1)), int(match.group(2))
    return None, None # For games without a valid score

# Apply the cleaning function
scores = History['Score'].apply(extract_clean_score)
History[['ScoreH', 'ScoreA']] = pd.DataFrame(scores.tolist(), index=History.index)

# Drop matches where the score couldn't be parsed (e.g. "Postponed" or "Suspended")
History = History.dropna(subset=['ScoreH', 'ScoreA'])

# Ensure they are integers and calculate total goals
History['ScoreH'] = History['ScoreH'].astype(int)
History['ScoreA'] = History['ScoreA'].astype(int)
History['But'] = History['ScoreH'] + History['ScoreA']

# 4. Result Logic using vectorized np.select (cleaner than nested np.where)
conditions = [
    (History['ScoreH'] == History['ScoreA']),
    (History['ScoreH'] > History['ScoreA'])
]
choices = ['Draw', 'Home']
History['Result'] = np.select(conditions, choices, default='Away')

# 5. Binary Outcome Columns (Vectorized is 10x faster than .apply)
History['Ds'] = (History['Result'] == 'Draw').astype(int)
History['Hs'] = (History['Result'] == 'Home').astype(int)
History['As'] = (History['Result'] == 'Away').astype(int)
History['DA'] = History['Result'].isin(['Draw', 'Away']).astype(int)
History['DH'] = History['Result'].isin(['Draw', 'Home']).astype(int)
History['HA'] = History['Result'].isin(['Home', 'Away']).astype(int)

# 6. Both Teams to Score (LDEM) logic
History['LDEMs'] = ((History['ScoreH'] > 0) & (History['ScoreA'] > 0)).astype(int)
History['LDEMPs'] = (History['LDEMs'] == 0).astype(int)

# 7. Combined Markets (Complex Logic)
History['DALDEMs'] = (History['LDEMs'] & History['DA']).astype(int)
History['DHLDEMs'] = (History['LDEMs'] & History['DH']).astype(int)
History['HALDEMs'] = (History['LDEMs'] & History['HA']).astype(int)
History['DALDEMPs'] = (History['LDEMPs'] & History['DA']).astype(int)
History['DHLDEMPs'] = (History['LDEMPs'] & History['DH']).astype(int)
History['HALDEMPs'] = (History['LDEMPs'] & History['HA']).astype(int)

# 8. Date Conversion - Handling the ESPN format "Sat, 10 Aug 2024"
# Using pd.to_datetime is safer than strptime inside an apply
History['Date'] = pd.to_datetime(History['Date'], format="%a, %d %b %Y", errors='coerce')

# Drop any rows where the date failed to parse (optional)
History = History.dropna(subset=['Date'])

# 9. Create unique IdGame and sort
History["IdGame"] = History["Date"].dt.strftime('%Y-%m-%d') + History["Home"] + History["Away"]
History = History.sort_values('Date', ascending=False)

In [22]:
import os

# --- Save History ---
file_path = "History.csv"

# 1. Load existing data or create empty DF
if os.path.exists(file_path):
    csv_df = pd.read_csv(file_path)
    # Ensure ID is string for robust comparison
    csv_df['IdGame'] = csv_df['IdGame'].astype(str)
else:
    # If file doesn't exist, create an empty one with History's columns
    csv_df = pd.DataFrame(columns=History.columns)

# 2. Identify brand new matches (Fast Method)
# Using .isin() is significantly faster than the 'bool' function with .contains()
new_ids = History['IdGame'].astype(str)
existing_ids = csv_df['IdGame'].unique()

is_new = ~new_ids.isin(existing_ids)

# 3. Concatenate and Cleanup
if is_new.any():
    new_data = History[is_new]
    
    # Merge new matches into the existing database
    updated_csv = pd.concat([csv_df, new_data], ignore_index=True)
    
    # Standardize dates
    updated_csv['Date'] = pd.to_datetime(updated_csv['Date'])
    
    # Remove any accidental duplicates and sort by newest first
    updated_csv = updated_csv.drop_duplicates(subset=['IdGame'])
    updated_csv = updated_csv.sort_values(by='Date', ascending=False)
    
    # Save to file
    updated_csv.to_csv(file_path, index=False, encoding='utf-8')
    print(f"✅ Successfully added {len(new_data)} new historical matches to {file_path}.")
else:
    print("ℹ️ No new matches to add. History.csv is already up to date.")

✅ Successfully added 2 new historical matches to History.csv.
